In [4]:
import tensorflow as tf
import os 
import pandas as pd
import numpy as np

In [5]:
df = pd.read_csv('train.csv')

In [6]:
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [14]:
df.iloc[5]['comment_text']

'"\n\nCongratulations from me as well, use the tools well. \xa0· talk "'

In [15]:
from tensorflow.keras.layers import TextVectorization

In [16]:
#Features
X = df['comment_text']
#Target values
y=df[df.columns[2:]].values

In [19]:
max_features = 200000

In [20]:
vectorizer = TextVectorization(max_tokens=max_features,
                               output_sequence_length=1800,
                               output_mode='int')

In [21]:
vectorizer.adapt(X.values)

In [24]:
vectorizer('Hi, how are you today')

<tf.Tensor: shape=(1800,), dtype=int64, numpy=array([171,  73,  20, ...,   0,   0,   0], dtype=int64)>

In [25]:
vectorized_text = vectorizer(X.values)

In [26]:
vectorized_text

<tf.Tensor: shape=(159571, 1800), dtype=int64, numpy=
array([[  645,    76,     2, ...,     0,     0,     0],
       [    1,    54,  2489, ...,     0,     0,     0],
       [  425,   441,    70, ...,     0,     0,     0],
       ...,
       [32445,  7392,   383, ...,     0,     0,     0],
       [    5,    12,   534, ...,     0,     0,     0],
       [    5,     8,   130, ...,     0,     0,     0]], dtype=int64)>

In [27]:
df = tf.data.Dataset.from_tensor_slices((vectorized_text,y))
df = df.cache()
df=df.shuffle(160000)
df=df.batch(16)
df=df.prefetch(8)

In [32]:
train = df.take(int(len(df)*.7))
val=df.skip(int(len(df)*.7)).take(int(len(df)*.2))
test=df.skip(int(len(df)*.9)).take(int(len(df)*.1))

In [39]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Bidirectional, Dense, Embedding

In [40]:
model =Sequential()
#Embedding layer(# of embedding,# of values long)
model.add(Embedding(max_features+1,32))
#Bidirectional(LSTM layer(# of units,lstm requires tanh activation))
model.add(Bidirectional(LSTM(32,activation='tanh')))
#Feature extractor Fully connected layers
model.add(Dense(128,activation='relu'))
model.add(Dense(256,activation='relu'))
model.add(Dense(128,activation='relu'))
#Final layer(units=total number of targets)
model.add(Dense(6,activation='sigmoid'))

In [41]:
model.compile(loss='BinaryCrossentropy',optimizer='Adam')

In [42]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          6400032   
                                                                 
 bidirectional (Bidirectiona  (None, 64)               16640     
 l)                                                              
                                                                 
 dense (Dense)               (None, 128)               8320      
                                                                 
 dense_1 (Dense)             (None, 256)               33024     
                                                                 
 dense_2 (Dense)             (None, 128)               32896     
                                                                 
 dense_3 (Dense)             (None, 6)                 774       
                                                        

In [43]:
history = model.fit(train,epochs=1,validation_data=val)

  21/6981 [..............................] - ETA: 1:46:48 - loss: 0.4975

KeyboardInterrupt: 